## picoCTF 2019 Web Challenges

### Insp3ct0r - Points: 50

Kishor Balan tipped us off that the following code may need inspection: https://2019shell1.picoctf.com/problem/63975/ or http://2019shell1.picoctf.com:63975

First we will import the functionality we will need for this challenge. Requests will allow us to easily talk to the webserver and BeautifulSoup will allow us to work with the HTML code that is the webserver provides us.

In [1]:
import requests
from bs4 import BeautifulSoup as bs
from bs4 import Comment

We will create a session object, get our URL for the challenge, initialize an empty flag, and get the main page.

In [2]:
s = requests.session()
url = 'https://2019shell1.picoctf.com/problem/63975/'
flag = ""
r = s.get(url)

Now that we have the base webpage's source code in our request object `r`, we will build see if there is a flag in our source.

In [3]:
[line for line in r.text.split('\n') if 'flag' in line]

['\t<!-- Html is neat. Anyways have 1/3 of the flag: picoCTF{tru3_d3 -->']

So we have a comment with that tells us the flag will be in parts. Seems that we have part 1. Lets creat a regular expression based on this, that may help us find the rest of the pieces.

In [4]:
from re import compile
patt = compile('flag: ([\w{}_?]*) ')

We have the source code for the site, lets build a BeautifulSoup object so we can grab that comment.

In [5]:
soup = bs(r.text,features="html.parser")

Since our regular expression has the a capture group defined, we will just grab that chunk from our comments and add them to our flag.

In [6]:
m = patt.findall(r.text)
if m:
    flag += m[0]

We will now grab all the hyperlinks from our `soup` object, and see if we have any pieces of the flag there.

In [7]:
htmls = ""
for link in soup.findAll('link'):
    r = s.get(url + link['href'])
    htmls += r.text
[line for line in htmls.split('\n') if 'flag' in line]

["/* You need CSS to make pretty pages. Here's part 2/3 of the flag: t3ct1ve_0r_ju5t */"]

Now we will grab that piece of the flag and add it to `flag` variable.

In [8]:
for link in soup.findAll('link'):
    r = s.get(url + link['href'])
    m = patt.findall(r.text)
    if m:
        flag += m[0]

Another place to look are any javascript files that might exist. So we will check anything from a `script` tag.

In [9]:
js = ""
for script in soup.findAll('script'):
    r = s.get(url + script['src'])
    js += r.text
[line for line in js.split('\n') if 'flag' in line]

['/* Javascript sure is neat. Anyways part 3/3 of the flag: _lucky?d3db9182} */']

Now we will grab that final piece of the flag and append to our `flag` variable

In [10]:
for script in soup.findAll('script'):
    r = s.get(url + script['src'])
    m2 = patt.findall(r.text)
    if m2:
        flag += m2[0]

And now we can check out final flag. Looks good to me.

In [11]:
flag

'picoCTF{tru3_d3t3ct1ve_0r_ju5t_lucky?d3db9182}'